# Практическое задание к уроку "Создание признакового пространства"

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

### Загрузим данные

In [2]:
combine_df = pd.read_pickle("combine_df.pkl")
combine_df.head(5)

,id,label,tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thank, lyft, credit, use, cause, offer, wheel..."
2,3,0.0,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"
3,4,0.0,model love you take with you all the time in ur,"[model, love, you, take, with, you, all, the, ...","[model, love, take, time, ur]","[model, love, take, time, ur]","[model, love, take, time, ur]"
4,5,0.0,factsguide society now motivation,"[factsguide, society, now, motivation]","[factsguide, society, motivation]","[factsguid, societi, motiv]","[factsguide, society, motivation]"


### Задание 1. Создайте мешок слов с помощью sklearn.feature_extraction.text.CountVectorizer.fit_transform(). 
- Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.


In [3]:
combine_df['tweet_stemmed_string'] = combine_df['tweet_stemmed'].apply(lambda tokens: ' '.join(tokens))
combine_df['tweet_lemmatized_string'] = combine_df['tweet_lemmatized'].apply(lambda tokens: ' '.join(tokens))

In [4]:
df = combine_df[['id', 'tweet_stemmed_string', 'tweet_lemmatized_string', 'label']]
df.head()

,id,tweet_stemmed_string,tweet_lemmatized_string,label
0,1,father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunct...,0.0
1,2,thank lyft credit use caus offer wheelchair va...,thank lyft credit use cause offer wheelchair v...,0.0
2,3,bihday majesti,bihday majesty,0.0
3,4,model love take time ur,model love take time ur,0.0
4,5,factsguid societi motiv,factsguide society motivation,0.0


In [5]:
df['label'].value_counts()

0.0    29720
1.0     2242
Name: label, dtype: int64

In [6]:
np.isnan(df['label']).sum()

17197

In [7]:
df['label'] = df['label'].apply(lambda y: 0 if np.isnan(y) else y)

<ipython-input-7-ea72096584b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].apply(lambda y: 0 if np.isnan(y) else y)


In [8]:
df.head()

,id,tweet_stemmed_string,tweet_lemmatized_string,label
0,1,father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunct...,0.0
1,2,thank lyft credit use caus offer wheelchair va...,thank lyft credit use cause offer wheelchair v...,0.0
2,3,bihday majesti,bihday majesty,0.0
3,4,model love take time ur,model love take time ur,0.0
4,5,factsguid societi motiv,factsguide society motivation,0.0


- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'. 

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(ngram_range=(1, 1), 
                                   analyzer='word', 
                                   binary=False, 
                                   stop_words="english", 
                                   max_df=0.9, 
                                   max_features=1000,)

- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью CountVectorizer.get_feature_names().

для stemmed count

In [10]:
bag_of_words_stemmed = count_vectorizer.fit_transform(df['tweet_stemmed_string'])
feature_names = count_vectorizer.get_feature_names()
stemmed_count_vectorizer_matrix = pd.DataFrame(bag_of_words_stemmed.toarray(), columns = feature_names)
stemmed_count_vectorizer_matrix.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


для lemmatized count

In [11]:
bag_of_words_lemmatized = count_vectorizer.fit_transform(df['tweet_lemmatized_string'])
feature_names = count_vectorizer.get_feature_names()
lemmatized_count_vectorizer_matrix = pd.DataFrame(bag_of_words_lemmatized.toarray(),
                                                  columns = feature_names)
lemmatized_count_vectorizer_matrix.head()

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Задание 2. Создайте мешок слов с помощью sklearn.feature_extraction.text.TfidfVectorizer.fit_transform(). 

- Игнорируем слова, частота которых в документе строго превышает порог 0.9 с помощью max_df.
- Ограничим количество слов, попадающий в мешок, с помощью max_features = 1000.
- Исключим стоп-слова с помощью stop_words='english'.

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 1), 
                                   analyzer='word', 
                                   binary=False, 
                                   max_df=0.9, 
                                   max_features=1000,
                                   stop_words='english')

- Применим его к 'tweet_stemmed' и 'tweet_lemmatized' отдельно.

- Отобразим Bag-of-Words модель как DataFrame. columns необходимо извлечь с помощью TfidfVectorizer.get_feature_names().

для stemmed tfidf

In [13]:
bag_of_words_stemmed = tfidf_vectorizer.fit_transform(df['tweet_stemmed_string'])
feature_names = tfidf_vectorizer.get_feature_names()
stemmed_tfidf_vectorizer_matrix = pd.DataFrame(bag_of_words_stemmed.toarray(),
                                  columns=feature_names)
stemmed_tfidf_vectorizer_matrix.head()

,abl,absolut,accept,account,act,action,actor,actual,ad,adapt,...,yeah,year,yesterday,yo,yoga,york,young,youtub,yr,yummi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


для lemmatized tfidf

In [14]:
bag_of_words_lemmatized = tfidf_vectorizer.fit_transform(df['tweet_lemmatized_string'])
feature_names = tfidf_vectorizer.get_feature_names()
lemmatized_tfidf_vectorizer_matrix = pd.DataFrame(bag_of_words_lemmatized.toarray(),
                                  columns=feature_names)
lemmatized_tfidf_vectorizer_matrix.head()

,able,absolutely,accept,account,act,action,actor,actually,adapt,add,...,yesterday,yo,yoga,york,young,youth,youtube,yr,yrs,yummy
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Задание 3. Проверьте ваши векторайзеры на корпусе который использовали на вебинаре, составьте таблицу метод векторизации и скор который вы получили (в методах векторизации по изменяйте параметры что бы добиться лучшего скора) обратите внимание как падает/растёт скор при уменьшении количества фичей, и изменении параметров, 

In [15]:
from sklearn import model_selection, preprocessing, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

In [16]:
X_stem_count = stemmed_count_vectorizer_matrix.astype('float32')
X_stem_tfidf = stemmed_tfidf_vectorizer_matrix.astype('float32')

X_lem_count = lemmatized_count_vectorizer_matrix.astype('float32')
X_lem_tfidf = lemmatized_tfidf_vectorizer_matrix.astype('float32')

y = df['label'].astype('float32')

results = []
cases = [['stem count', X_stem_count], ['stem tfidf', X_stem_tfidf], ['lem count', X_lem_count], ['lem tfidf', X_lem_tfidf]]

In [17]:
for case in cases:
    X_train, X_test, y_train, y_test = model_selection.train_test_split(case[1], y)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append([
        case[0], 
        mean_squared_error(y_test, y_pred), 
        mean_absolute_error(y_test, y_pred),
        median_absolute_error(y_test, y_pred), 
        r2_score(y_test, y_pred)
    ])

In [18]:
results

[['stem count', 0.037623715, 0.084559284, 0.034380622, 0.19739614672384798],
 ['stem tfidf', 0.034505844, 0.08229496, 0.029802574, 0.22882582361159987],
 ['lem count', 132.48676, 0.21605484, 0.036545515, -3125.2719999897663],
 ['lem tfidf', 41995.35, 1.9291726, 0.029538123, -1023236.0753148039]]

### так же попробуйте применить к векторайзерам PCA для сокращения размерности посмотрите на качество сделайте выводы

In [19]:
from sklearn import decomposition 

In [20]:
max_numbers_of_features = [500, 3000]

In [21]:
for max_number_of_features in max_numbers_of_features:
    tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   analyzer='word', 
                                   binary=False, 
                                   max_df=0.9, 
                                   max_features=max_number_of_features,
                                   stop_words='english')

    X_lemmatized_tfidf = lemmatized_tfidf_vectorizer_matrix.astype('float32')

    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_lemmatized_tfidf, y)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append([
        f'{max_number_of_features} features lemmatized tfidf',
        mean_squared_error(y_test, y_pred), 
        mean_absolute_error(y_test, y_pred),
        median_absolute_error(y_test, y_pred), 
        r2_score(y_test, y_pred)
    ])

In [22]:
results

[['stem count', 0.037623715, 0.084559284, 0.034380622, 0.19739614672384798],
 ['stem tfidf', 0.034505844, 0.08229496, 0.029802574, 0.22882582361159987],
 ['lem count', 132.48676, 0.21605484, 0.036545515, -3125.2719999897663],
 ['lem tfidf', 41995.35, 1.9291726, 0.029538123, -1023236.0753148039],
 ['500 features lemmatized tfidf',
  0.03318647,
  0.07966132,
  0.029594153,
  0.19139550979578346],
 ['3000 features lemmatized tfidf',
  0.03437247,
  0.08214512,
  0.03028313,
  0.21364391987409614]]

In [23]:
pca_n_components_numbers = [50, 100]

In [24]:
for pca_n_components in pca_n_components_numbers:
    pca = decomposition.PCA(n_components=pca_n_components)
    pca.fit(X_lemmatized_tfidf)

    X_lemmatized_tfidf_pca = pca.transform(X_lemmatized_tfidf)
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X_lemmatized_tfidf_pca, y)

    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    results.append([
        f'PCA {pca_n_components} features lemmatized tfidf',
        mean_squared_error(y_test, y_pred), 
        mean_absolute_error(y_test, y_pred),
        median_absolute_error(y_test, y_pred), 
        r2_score(y_test, y_pred)])

In [25]:
result_df = pd.DataFrame(results, columns=['case', 'mse', 'mae', 'median', 'r2 score'])

In [26]:
result_df

,case,mse,mae,median,r2 score
0,stem count,0.037624,0.084559,0.034381,1.973961e-01
1,stem tfidf,0.034506,0.082295,0.029803,2.288258e-01
2,lem count,132.486755,0.216055,0.036546,-3.125272e+03
3,lem tfidf,41995.351562,1.929173,0.029538,-1.023236e+06
4,500 features lemmatized tfidf,0.033186,0.079661,0.029594,1.913955e-01
5,3000 features lemmatized tfidf,0.034372,0.082145,0.030283,2.136439e-01
6,PCA 50 features lemmatized tfidf,0.041267,0.089426,0.051518,7.159707e-02
7,PCA 100 features lemmatized tfidf,0.039927,0.088057,0.042172,1.105942e-01
